<img src="../Pierian-Data-Logo.PNG">
<br>
<strong><center>Copyright 2019. Created by Jose Marcial Portilla.</center></strong>

# What is CUDA?

Most people confuse CUDA for a language or maybe an API. It is not.

It’s more than that. CUDA is a parallel computing platform and programming model that makes using a GPU for general purpose computing simple and elegant. The developer still programs in the familiar C, C++, Fortran, or an ever expanding list of supported languages, and incorporates extensions of these languages in the form of a few basic keywords.

These keywords let the developer express massive amounts of parallelism and direct the compiler to the portion of the application that maps to the GPU.

# How do I install PyTorch for GPU?

Refer to video, its dependent on whether you have an NVIDIA GPU card or not.

# How do I know if I have CUDA available?

In [1]:
import torch
torch.cuda.is_available()
# True

True

# Using GPU and CUDA

We've provided 2 versions of our yml file, a GPU version and a CPU version. To use GPU, you need to either manually create a virtual environment, please watch the video related to this lecture, as not every computer can run GPU, you need CUDA and an NVIDIA GPU.

In [2]:
## Get Id of default device
torch.cuda.current_device()

0

In [3]:
# 0
torch.cuda.get_device_name(0) # Get name device with ID '0'

'NVIDIA GeForce RTX 3080'

In [4]:
# Returns the current GPU memory usage by 
# tensors in bytes for a given device
torch.cuda.memory_allocated()

0

In [6]:
# Returns the current GPU memory managed by the
# caching allocator in bytes for a given device
torch.cuda.memory_reserved()

0

# Using CUDA instead of CPU

In [7]:
# CPU
a = torch.FloatTensor([1.,2.])

In [8]:
a

tensor([1., 2.])

In [10]:
a.device

device(type='cpu')

In [11]:
# GPU
a = torch.FloatTensor([1., 2.]).cuda()

In [12]:
a.device

device(type='cuda', index=0)

In [13]:
torch.cuda.memory_allocated()

512

## Sending Models to GPU

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
class Model(nn.Module):
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)    # input layer
        self.fc2 = nn.Linear(h1, h2)            # hidden layer
        self.out = nn.Linear(h2, out_features)  # output layer
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [16]:
torch.manual_seed(32)
model = Model()

In [17]:
# From the discussions here: discuss.pytorch.org/t/how-to-check-if-model-is-on-cuda
next(model.parameters()).is_cuda

False

In [18]:
gpumodel = model.cuda()

In [19]:
next(gpumodel.parameters()).is_cuda

True

In [26]:
import os
import pandas as pd
from sklearn.datasets import load_iris

# Load the Iris dataset
iris = load_iris()

# Convert the data into a DataFrame
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target

# Specify the directory where you want to save the dataset
save_dir = '../DATA'
os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist

# Save the dataset as a CSV file
iris_df.to_csv(os.path.join(save_dir, 'iris.csv'), index=False)

print(f"Iris dataset saved to {save_dir}/iris.csv")

Iris dataset saved to ../DATA/iris.csv


In [27]:


df = pd.read_csv('../DATA/iris.csv')
X = df.drop('target',axis=1).values
y = df['target'].values
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=33)

## Convert Tensors to .cuda() tensors

In [28]:
X_train = torch.FloatTensor(X_train).cuda()
X_test = torch.FloatTensor(X_test).cuda()
y_train = torch.LongTensor(y_train).cuda()
y_test = torch.LongTensor(y_test).cuda()

In [32]:
trainloader = DataLoader(X_train, batch_size=60, shuffle=True, pin_memory=True)
testloader = DataLoader(X_test, batch_size=60, shuffle=False, pin_memory=True)

In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [35]:
import time
epochs = 100
losses = []
start = time.time()
for i in range(epochs):
    i+=1
    y_pred = gpumodel.forward(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss)
    
    # a neat trick to save screen space:
    if i%10 == 0:
        print(f'epoch: {i:2}  loss: {loss.item():10.8f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print(f'TOTAL TRAINING TIME: {time.time()-start}')

epoch: 10  loss: 0.05378844
epoch: 20  loss: 0.05303511
epoch: 30  loss: 0.05232612
epoch: 40  loss: 0.05165950
epoch: 50  loss: 0.05103232
epoch: 60  loss: 0.05044188
epoch: 70  loss: 0.04988527
epoch: 80  loss: 0.04935985
epoch: 90  loss: 0.04886319
epoch: 100  loss: 0.04839361
TOTAL TRAINING TIME: 0.1378486156463623
